# Satellite Image Segmentation using SAM (Segment Anything Model)

In [ ]:
# Step 1: Install the necessary packages
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python matplotlib

### Step 2: Download and Load a Satellite Image

In [ ]:
import cv2
import matplotlib.pyplot as plt

# You can replace the below image URL with your satellite image or dataset path
image_url = "https://upload.wikimedia.org/wikipedia/commons/8/84/Nasa_landcover_Large.png"
image_path = "satellite_image.jpg"

# Downloading the image
import urllib.request
urllib.request.urlretrieve(image_url, image_path)

# Load the image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Display the image
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis('on')
plt.show()

### Step 3: Load SAM and Set the Image for Prediction

In [ ]:
import torch
from segment_anything import SamPredictor, sam_model_registry

# Load SAM model
sam = sam_model_registry["default"](checkpoint="sam_vit_h_4b8939.pth")  # Ensure you have downloaded the checkpoint
device = "cuda" if torch.cuda.is_available() else "cpu"
sam.to(device)

# Create the predictor
predictor = SamPredictor(sam)

# Set the image for prediction
predictor.set_image(image)

### Step 4: Define Points for Segmentation

In [ ]:
import numpy as np

# Define a point on the image for segmentation (x, y)
input_point = np.array([[800, 500]])  # You can modify the coordinates based on your image
input_label = np.array([1])  # 1 for positive label

# Show the selected point on the image
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.scatter(input_point[:, 0], input_point[:, 1], color='green', marker='*', s=300, edgecolor='white')
plt.axis('on')
plt.show()

### Step 5: Perform Segmentation

In [ ]:
# Perform segmentation with SAM
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True
)

# Visualize the mask on the satellite image
for i, (mask, score) in enumerate(zip(masks, scores)):
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * np.array([30/255, 144/255, 255/255, 0.6]).reshape(1, 1, -1)
    plt.imshow(mask_image)
    plt.title(f"Mask {i+1}, Score: {score:.3f}", fontsize=18)
    plt.axis('off')
    plt.show()

### Step 6: Perform Advanced Segmentation with Boxes

In [ ]:
# Define a bounding box for segmentation
input_box = np.array([600, 400, 1200, 800])

# Perform segmentation with the box
masks, _, _ = predictor.predict(
    point_coords=None,
    point_labels=None,
    box=input_box[None, :],
    multimask_output=False
)

# Visualize the result
plt.figure(figsize=(10, 10))
plt.imshow(image)
h, w = masks[0].shape[-2:]
mask_image = masks[0].reshape(h, w, 1) * np.array([30/255, 144/255, 255/255, 0.6]).reshape(1, 1, -1)
plt.imshow(mask_image)
plt.gca().add_patch(plt.Rectangle((input_box[0], input_box[1]), input_box[2] - input_box[0], input_box[3] - input_box[1],
                                  edgecolor='green', facecolor='none', lw=2))
plt.axis('off')
plt.show()